# Advent of Code 2022

## Day 25: Full of Hot Air

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

Today, code is mess.

Finished 12:04 p.m. Xmas day.

I'm off to play video games.

In [ ]:
from typing import Optional, Iterator, Any, Callable

In [ ]:
def read(filename: str, limit: Optional[int] = None) -> Iterator[str]:
    with open(filename) as file:
        count = 0
        for line in file:
            line = line.strip()
            yield line
            count += 1
            if limit is not None and count >= limit:
                break

In [ ]:
LOOKUP = {
    '=' : -2,
    '-' : -1,
    '0' : 0,
    '1' : 1,
    '2' : 2,
    -2 : '=',
    -1 : '-',
    0 : '0',
    1 : '1',
    2 : '2'
}

def read_ints(filename: str, limit: Optional[int] = None) -> Iterator[tuple[str, list[int]]]:
    for line in read(filename, limit):
        yield line, [LOOKUP[e] for e in reversed(list(line))]

def stringify(snafu: list[int]) -> str:
    rv = ''
    for e in snafu:
        rv = LOOKUP[e] + rv
    return rv

In [ ]:
def decode(snafu: list[int]) -> int:
    multiplier = 1
    rv = 0
    for value in snafu:
        rv += value * multiplier
        multiplier *= 5
    return rv

In [ ]:
def to_proxy(snafu: list[int]) -> int:
    multiplier = 1
    rv = 0
    for e in snafu:
        rv += multiplier * (e + 2)
        multiplier *= 5
    return rv

def from_proxy(proxy: int) -> list[int]:
    rv = []
    divider = 5
    while proxy != 0:
        assert proxy > 0
        current = proxy % divider
        proxy = proxy // divider
        rv += [current - 2]
    return rv

In [ ]:
INPUT_DATA = 'data/input25.txt'

total = 0
print(f'{"SNAFU":>25}{"Decimal":>25}')
for line, ints in read_ints(INPUT_DATA):
    print(f'{line:>25}{decode(ints):>25}')
    total += decode(ints)

print()
print(f'Total = {total}')

In [ ]:
def num_digits(number: int) -> int:
    rv = [2]
    while decode(rv) < number:
        rv = [2] + rv
    return len(rv)

In [ ]:
def sgn(x):
    assert (type(x) == int), f'called sgn({x}) with type {type(x)}'
    if x < 0:
        return -1
    if x > 0:
        return 1
    return 0


def __binary_search(i: int, k: int, f_i: Any, f_k: Any, f: Callable[[int], Any]) -> Optional[int]:
    assert (type(i) == int), f'called __binary_search(... i={i} ...)'
    assert (type(k) == int), f'called __binary_search(... k={k} ...)'
    assert (type(f_i) == int), f'called __binary_search(... f_i={f_i} ...)'
    assert (type(f_k) == int), f'called __binary_search(... f_k={f_k} ...)'
    if f_i == 0:
        return i
    elif f_k == 0:
        return k
    if sgn(f_i) == sgn(f_k):
        return None
    if i >= k:
        return None
    if i == k - 1:
        return None
    j = (i + k) // 2
    assert i < j < k
    f_j = f(j)
    if f_j == 0:
        return j
    if sgn(f_i) == sgn(f_j):
        return __binary_search(j, k, f_j, f_k, f)
    return __binary_search(i, j, f_i, f_j, f)


def binary_search(i: int, j: int, f: Callable[[int], Any]) -> Optional[int]:
    f_i = f(i)
    f_j = f(j)
    assert type(f_i) == int
    assert type(f_j) == int
    if sgn(f_i) == sgn(f_j):
        raise ValueError(f'f({i}) = {f_i}, f({j}) = {f_j}')
    return __binary_search(i, j, f(i), f(j), f)

In [ ]:
def proxy_to_decimal(proxy: int) -> int:
    assert (type(proxy) == int), f'called proxy_to_decimal({proxy}) with type {type(proxy)}'
    return decode(from_proxy(proxy))

def diff_total(total: int) -> Callable[[int], int]:
    def f(x):
        assert (type(x) == int), f'called f({x}) with type {type(x)}'
        return proxy_to_decimal(x) - total
    return f

d = num_digits(total)

lower = []
upper = [2] * d

lower_proxy = to_proxy(lower)
upper_proxy = to_proxy(upper)

proxy = binary_search(lower_proxy, upper_proxy, diff_total(total))

proxy

stringify(from_proxy(proxy))
